In [1]:
import time
from bs4 import BeautifulSoup
import pandas as pd
import math
from amazon_bot import Amazon_bot

In [2]:
url='https://www.amazon.com'
search_item='high-end laptop'
amazon_bot=Amazon_bot(url,search_item)


In [3]:
htmls=amazon_bot.get_html()

Error occurred during sending2 next key


In [4]:
def parse_pages():
    print(f"Total pages loaded: {len(htmls)}")
    products = []

    for html in htmls:
        if not html:
            continue
        soup = BeautifulSoup(html, 'html.parser')
        items = soup.find_all('div', class_='puisg-col-inner')
        for item in items:
            # Extract product name
            name = None
            h2 = item.find('h2')
            if h2:
                span = h2.find('span')
                if span and span.text.strip():
                    name = span.text.strip()
            else:
                span = item.find('span', class_='a-size-medium')
                if span and span.text.strip():
                    name = span.text.strip()
            # Extract price
            price = None
            price_span = item.find('span', class_='a-price-whole')
            if price_span and price_span.text.strip():
                price = price_span.text.strip()
            else:
                price_span = item.find('span', class_='a-color-base')
                if price_span and price_span.text.strip():
                    price = price_span.text.strip()
            # Only append if both name and price are found
            if name and price:
                products.append({'Name of the product': name, 'Price': price})

    df = pd.DataFrame(products)
    print(f"Total products found: {len(df)}")
    return df

# Call the function
price_data = parse_pages()

Total pages loaded: 20
Total products found: 290


In [21]:
price_data_df=pd.DataFrame(price_data,columns=['Name of the product','Price'])

In [22]:
price_data_df.head()

,Name of the product,Price
0,"NIMO 15.6 IPS FHD-Laptop, 16GB RAM 1TB SSD Int...",Small Business
1,"Lenovo V15 Laptop, 15.6"" FHD Display, AMD Ryze...",$450.00
2,"MSI Prestige 16 EVO 16"" QHD+ 165Hz Laptop: Int...",$925.60
3,"HP 255 G10 Business Laptop 250 Series (15.6"" F...",$599.98
4,MSI Prestige 13 AI+ Evo (2024) 13.3” 2.8K OLED...,"$1,160.25"


In [23]:
price_data_df['Price_num'] = price_data_df['Price'].apply(lambda s: s.split('$')[-1].replace(',', ''))
price_data_df['Price_num'] = pd.to_numeric(price_data_df['Price_num'], errors='coerce', downcast='integer')


In [24]:
price_data_df.head()

,Name of the product,Price,Price_num
0,"NIMO 15.6 IPS FHD-Laptop, 16GB RAM 1TB SSD Int...",Small Business,NaN
1,"Lenovo V15 Laptop, 15.6"" FHD Display, AMD Ryze...",$450.00,450.00
2,"MSI Prestige 16 EVO 16"" QHD+ 165Hz Laptop: Int...",$925.60,925.60
3,"HP 255 G10 Business Laptop 250 Series (15.6"" F...",$599.98,599.98
4,MSI Prestige 13 AI+ Evo (2024) 13.3” 2.8K OLED...,"$1,160.25",1160.25


In [25]:
price_data_df = price_data_df.dropna(subset=['Price_num'])

In [26]:
price_data_df.head()

,Name of the product,Price,Price_num
1,"Lenovo V15 Laptop, 15.6"" FHD Display, AMD Ryze...",$450.00,450.00
2,"MSI Prestige 16 EVO 16"" QHD+ 165Hz Laptop: Int...",$925.60,925.60
3,"HP 255 G10 Business Laptop 250 Series (15.6"" F...",$599.98,599.98
4,MSI Prestige 13 AI+ Evo (2024) 13.3” 2.8K OLED...,"$1,160.25",1160.25
5,"HP Victus 15 Gaming Laptop, 15.6"" FHD 144Hz Di...","1,269.",1269.00


In [27]:
price_data_df['laptops'] = price_data_df['Name of the product'].str.contains('laptop', case=False)
price_data_df = price_data_df[price_data_df['laptops']]

In [28]:
price_data_df=price_data_df.drop(columns=['Price','laptops'])

In [29]:
price_data_df=price_data_df.sort_values('Price_num', ignore_index=True, ascending=False)

In [30]:
price_data_df.to_csv('Amazonprice.csv',index=False)